# Building a RAG application from scratch

Let's start by loading the environment variables we need to use.

In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["***KEY***"] = os.getenv("***KEY***")
os.environ["***KEY***_ENV"] = os.getenv("***KEY***_ENV")
os.environ["***KEY***"] = os.getenv("***KEY***")


from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)


from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")



from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) are a simple way to define and reuse prompts.

In [4]:
from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant, trained to provide understandable and accurate information about pharmacogenomics and drugs.
You will base your responses on the context and information provided. Output both your answer and a score of how confident you are,
 along with citing the references. Also provide the source of the chunks of the documents used for generating response.
If the information related to the question is not in the context and or in the information provided in the prompt, 
you will say 'I don't know'.
You are not a healthcare provider and you will not provide medical care or make assumptions about treatment.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Let's start by loading the transcription in memory:

In [6]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

folder_path = "/home/akkey/Akshay/RAG/RAG_data/all_files"
jsondata = []
csvdata = []
pdfdocs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        doc = loader.load()
        pdfdocs.extend(doc)
    elif filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        loader = CSVLoader(file_path)
        data = loader.load()
        csvdata.extend(data)
    elif filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        loader = JSONLoader(file_path, jq_schema=".",json_lines=False,text_content=False)
        data = loader.load()
        jsondata.extend(data)

for doc in pdfdocs:
    doc.page_content = doc.page_content.replace('\t', ' ')


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
documents = text_splitter.split_documents(pdfdocs)
jsondocs = text_splitter.split_documents(jsondata)

documents += jsondocs + csvdata


Setting up Pinecone

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable `***KEY***`.

We can get a retriever directly from the vector store we created before: 

In [10]:
from langchain_pinecone import PineconeVectorStore

pinecone = PineconeVectorStore(embedding=embeddings, index_name="all-files")


retriever = pinecone.as_retriever()


from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever, question=RunnablePassthrough())


chain = setup | prompt | model | parser

In [23]:
import numpy as np
import pandas as pd
Questions = pd.read_csv("/home/akkey/Akshay/RAG/Questions.csv")
questions = Questions[Questions["Expected_response"].notna()]
quests = questions["Questions"]


cont = []
for quest in quests[]:
    cont.append([docs.page_content for docs in retriever.invoke(quest)])
questions["Context"] = cont

,Questions,Expected_response,Expected_source
0,"As part of my liver transplant, I take tacroli...",For a CYP3A5 extensive metabolizer. Increase s...,https://www.fda.gov/drugs/science-and-research...
1,"I'm Indian, and my family has a history of epi...",As per CPIC HLA-B*15:02 might be observed in 2...,https://www.pharmgkb.org/haplotype/PA165954769...
2,Allopurinol is what I take for gout. For my se...,There can be moderate interaction while consid...,https://insilico-cyp.charite.de/SuperCYPsPred/...
3,"Is it true that while taking clopidogrel, I sh...","Indeed, you should refrain from drinking grape...","Bailey DG, Malcolm J, Arnold O, Spence JD. Gra..."
4,Dizziness is one of the adverse reactions that...,"Dizziness drawiness,extreme thirst,fast, shall...",https://www.fda.gov/drugs/science-and-research...
...,...,...,...
167,What are the pharmacogenomic panel advised to ...,"G6PD, CYP2D6, UGT1A1",NaN
168,I have a 7 year old patient with HLA-B1502 pos...,Sodium Valproate,NaN
169,How does CYP2C9 affect anticoagulants mode of ...,NaN,https://www.pharmgkb.org/disease/PA166123298/l...
170,Which are the antidiabetic medications influen...,NaN,https://www.pharmgkb.org/chemical/PA164712711/...


In [ ]:
resp = []

n = len(questions)
i = 1

for question in questions:
    resp.append(chain.invoke(question))
    Questions["Response"] = resp + [np.nan] * (n - i)
    Questions.to_csv(os.path.join('Questions_with_updated_context_and_response.csv'))
    print(f"{(n-i)} questions left")
    i += 1

# Implementing Hybrid search and Reranking

### Ref: https://medium.com/@nadikapoudel16/advanced-rag-implementation-using-hybrid-search-reranking-with-zephyr-alpha-llm-4340b55fef22 

In [5]:
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import BM25Retriever, EnsembleRetriever

pinecone = PineconeVectorStore(embedding=embeddings, index_name="all-files")

retriever_vectordb = pinecone.as_retriever(search_kwargs={"k": 5})
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k =  5
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=compression_retriever, question=RunnablePassthrough())


chain = setup | prompt | model | parser

/home/akkey/Akshay/.local/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'documents' is not defined

In [25]:
import time
import numpy as np
import pandas as pd
Questions = pd.read_csv("/home/akkey/Akshay/RAG/Questions.csv")
questions = Questions[Questions["Expected_response"].notna()]
quests = questions["Questions"]

cont = []

n = len(quests)
i = 1

for quest in quests:
    cont.append([docs.page_content for docs in compression_retriever.invoke(quest)])
    questions["Context"] = cont + [np.nan] * (n - i)
    questions.to_csv("/home/akkey/Akshay/RAG/gen/gpt4o_gen/rag/updated_context/GPT_4_hybrid_search_reranking.csv")
    print(f"{(n-i)} questions left")
    i += 1
    time.sleep(60)

/tmp/ipykernel_8480/1886670316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


133 questions left


/tmp/ipykernel_8480/1886670316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


132 questions left


/tmp/ipykernel_8480/1886670316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


...

2 questions left


/tmp/ipykernel_8480/1886670316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


1 questions left


/tmp/ipykernel_8480/1886670316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


0 questions left


In [31]:
questions = pd.read_csv("/home/akkey/Akshay/RAG/gen/gpt4o_gen/rag/updated_context/GPT_4o_hybrid_search_reranking.csv")
questions

,Unnamed: 0,Questions,Expected_response,Expected_source,Context
0,0,"As part of my liver transplant, I take tacroli...",For a CYP3A5 extensive metabolizer. Increase s...,https://www.fda.gov/drugs/science-and-research...,['pharmacokinetics of tacrolimus in liver tran...
1,1,"I'm Indian, and my family has a history of epi...",As per CPIC HLA-B*15:02 might be observed in 2...,https://www.pharmgkb.org/haplotype/PA165954769...,['1. \nSukasem C et al. Spectrum of cutaneous ...
2,2,Allopurinol is what I take for gout. For my se...,There can be moderate interaction while consid...,https://insilico-cyp.charite.de/SuperCYPsPred/...,['Drugs: allopurinol\nGenes: HLA-B\nPhenotypes...
3,3,"Is it true that while taking clopidogrel, I sh...","Indeed, you should refrain from drinking grape...","Bailey DG, Malcolm J, Arnold O, Spence JD. Gra...",['Drug: clopidogrel\nGene1: CYP2C19\nPhenotype...
4,4,Dizziness is one of the adverse reactions that...,"Dizziness drawiness,extreme thirst,fast, shall...",https://www.fda.gov/drugs/science-and-research...,['genotypes. Arch Pharm Res 2006;29:525-33.\n9...
...,...,...,...,...,...
129,149,My patient has CYP2D6 poor metabolism and does...,CPIC guideline for ondansetron and CYP2D6 has ...,NaN,"[""Drug: ondansetron\nGene1: CYP2D6\nPhenotype1..."
130,166,What are side effects associated wtih DPYD poo...,5-FU has many adverse drug effects when used s...,NaN,"[""Drug: fluorouracil\nGene1: DPYD\nPhenotype1:..."
131,167,What are the pharmacogenomic panel advised to ...,"G6PD, CYP2D6, UGT1A1",NaN,['patients with Type 2 diabetes mellitus. Phar...
132,168,I have a 7 year old patient with HLA-B1502 pos...,Sodium Valproate,NaN,['Drug: Carbamazepine\nGene1: HLA-A(*31:01 gen...


In [32]:
quests =  questions["Questions"].to_list()
resp = []

n = len(quests)
i = 1

for quest in quests:
    resp.append(chain.invoke(quest))
    questions["Response"] = resp + [np.nan] * (n - i)
    questions.to_csv("/home/akkey/Akshay/RAG/gen/gpt4o_gen/rag/updated_context/GPT_4o_hybrid_search_reranking.csv")
    print(f"{(n-i)} questions left")
    i += 1
    time.sleep(60)

133 questions left
132 questions left
131 questions left
130 questions left
129 questions left
128 questions left
127 questions left
126 questions left
125 questions left
124 questions left
123 questions left
122 questions left
121 questions left
120 questions left
119 questions left
118 questions left
117 questions left
116 questions left
115 questions left
114 questions left
113 questions left
112 questions left
111 questions left
110 questions left
109 questions left
108 questions left
107 questions left
106 questions left
105 questions left
104 questions left
103 questions left
102 questions left
101 questions left
100 questions left
99 questions left
98 questions left
97 questions left
96 questions left
95 questions left
94 questions left
93 questions left
92 questions left
91 questions left
90 questions left
89 questions left
88 questions left
87 questions left
86 questions left
85 questions left
84 questions left
83 questions left
82 questions left
81 questions left
80 questions

# GPT-4, 3072 embeddings, hybrid search, reranking

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["***KEY***"] = os.getenv("***KEY***")
os.environ["***KEY***_ENV"] = os.getenv("***KEY***_ENV")
os.environ["***KEY***"] = os.getenv("***KEY***")


from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)


from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")



from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant, trained to provide understandable and accurate information about pharmacogenomics and drugs.
You will base your responses on the context and information provided. Output your answer in brief (not more than 2 lines) and also a score of how confident you are,
 along with citing the references. Also provide the source of the chunks of the documents used for generating response.
If the information related to the question is not in the context and or in the information provided in the prompt, 
you will say 'I don't know'.
You are not a healthcare provider and you will not provide medical care or make assumptions about treatment.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [18]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

folder_path = "/home/akkey/Akshay/RAG/RAG_data/all_files"
jsondata = []
csvdata = []
pdfdocs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        doc = loader.load()
        pdfdocs.extend(doc)
    elif filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        loader = CSVLoader(file_path)
        data = loader.load()
        csvdata.extend(data)
    elif filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        loader = JSONLoader(file_path, jq_schema=".",json_lines=False,text_content=False)
        data = loader.load()
        jsondata.extend(data)

for doc in pdfdocs:
    doc.page_content = doc.page_content.replace('\t', ' ')


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
documents = text_splitter.split_documents(pdfdocs)
jsondocs = text_splitter.split_documents(jsondata)

documents += jsondocs + csvdata

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x73325ec4d810>>
Traceback (most recent call last):
  File "/home/akkey/Akshay/.local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [24]:
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers import BM25Retriever, EnsembleRetriever

pinecone = PineconeVectorStore(embedding=embeddings, index_name="all-files")

retriever_vectordb = pinecone.as_retriever(search_kwargs={"k": 25})
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k =  25
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=compression_retriever, question=RunnablePassthrough())


chain = setup | prompt | model | parser

#### Generating the responses

In [13]:
import time
import numpy as np
import pandas as pd
Questions = pd.read_csv("/home/akkey/Akshay/RAG/Questions.csv")
questions = Questions[Questions["Expected_response"].notna()]
quests = questions["Questions"].to_list()

cont = []
resp = []

n = len(quests)
i = 1

for quest in quests:
    cont.append([docs.page_content for docs in compression_retriever.invoke(quest)])
    questions["Context"] = cont + [np.nan] * (n - i)
    print(f"{(n-i)} contexts left")
    time.sleep(60)
    resp.append(chain.invoke(quest))
    questions["Response"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(60)
    questions.to_csv("/home/akkey/Akshay/RAG/gen/gpt-4/rag/gpt4_large_embedding_hybrid_search_reranking.csv")
    print(f"{(n-i)} questions left")
    i += 1

/tmp/ipykernel_95681/3928612843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


133 contexts left


/tmp/ipykernel_95681/3928612843.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Response"] = resp + [np.nan] * (n - i)


133 responses left
133 questions left


/tmp/ipykernel_95681/3928612843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


...

/tmp/ipykernel_95681/3928612843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


49 contexts left


/tmp/ipykernel_95681/3928612843.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Response"] = resp + [np.nan] * (n - i)


49 responses left
49 questions left


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [25]:
len(cont)

87

In [26]:
len(resp)

87

In [27]:
i

88

In [28]:
len(quests[87:]) + 87 - i

46

In [29]:
for quest in quests[87:]:
    cont.append([docs.page_content for docs in compression_retriever.invoke(quest)])
    questions["Context"] = cont + [np.nan] * (n - i)
    print(f"{(n-i)} contexts left")
    time.sleep(60)
    resp.append(chain.invoke(quest))
    questions["Response"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(60)
    questions.to_csv("/home/akkey/Akshay/RAG/gen/gpt-4/rag/gpt4_large_embedding_hybrid_search_reranking_2.csv")
    print(f"{(n-i)} questions left")
    i += 1

/tmp/ipykernel_95681/3198995113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


46 contexts left


/tmp/ipykernel_95681/3198995113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Response"] = resp + [np.nan] * (n - i)


46 responses left
46 questions left


/tmp/ipykernel_95681/3198995113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


...

1 responses left
1 questions left


/tmp/ipykernel_95681/3198995113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Context"] = cont + [np.nan] * (n - i)


0 contexts left


/tmp/ipykernel_95681/3198995113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["Response"] = resp + [np.nan] * (n - i)


0 responses left
0 questions left


# Using Document Intelligence

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["***KEY***"] = os.getenv("***KEY***")
os.environ["***KEY***_ENV"] = os.getenv("***KEY***_ENV")
os.environ["***KEY***"] = os.getenv("***KEY***")


from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")



from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant, trained to provide understandable and accurate information about pharmacogenomics and drugs.
You will base your responses on the context and information provided. Output your answer in brief (not more than 2 lines) and also a score of how confident you are,
 along with citing the references. Also provide the source of the chunks of the documents used for generating response.
If the information related to the question is not in the context and or in the information provided in the prompt, 
you will say 'I don't know'.
You are not a healthcare provider and you will not provide medical care or make assumptions about treatment.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

folder_path = "/home/akkey/Akshay/RAG/RAG_data/all_files"
jsondata = []
csvdata = []
pdfdocs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        doc = loader.load()
        pdfdocs.extend(doc)
    elif filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        loader = CSVLoader(file_path)
        data = loader.load()
        csvdata.extend(data)
    elif filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        loader = JSONLoader(file_path, jq_schema=".",json_lines=False,text_content=False)
        data = loader.load()
        jsondata.extend(data)

for doc in pdfdocs:
    doc.page_content = doc.page_content.replace('\t', ' ')


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
documents = text_splitter.split_documents(pdfdocs)
jsondocs = text_splitter.split_documents(jsondata)

documents += jsondocs + csvdata

/home/akkey/Akshay/.local/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [13]:
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import BM25Retriever, EnsembleRetriever

pinecone = PineconeVectorStore(embedding=embeddings, index_name="doc-intel")

retriever_vectordb = pinecone.as_retriever(search_kwargs={"k": 25})
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k =  25
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")


compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=compression_retriever, question=RunnablePassthrough())


chain = setup | prompt | model | parser

In [15]:
import time
import numpy as np
import pandas as pd
Questions = pd.read_csv("/home/akkey/Akshay/RAG/Questions.csv")
questions = Questions[Questions["Expected_response"].notna()][:58]
quests = questions["Questions"].to_list()

cont = []
resp = []

n = len(quests)
i = 1

for quest in quests:
    context = [docs.page_content for docs in compression_retriever.invoke(quest)]
    cont.append(context)
    questions["Context"] = cont + [np.nan] * (n - i)
    print(f"{(n-i)} contexts left")
    time.sleep(60)
    response = chain.invoke(quest)
    resp.append(response)
    questions["Response"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(60)
    questions.to_csv("/home/akkey/Akshay/RAG/gen/gpt4o/rag/large_embedding_short_ans_with_reranking_short_responses.csv")
    print(f"{(n-i)} questions left")
    i += 1

57 contexts left
57 responses left
57 questions left
56 contexts left
56 responses left
56 questions left
55 contexts left
55 responses left
55 questions left
54 contexts left
54 responses left
54 questions left
53 contexts left
53 responses left
53 questions left
52 contexts left
52 responses left
52 questions left
51 contexts left
51 responses left
51 questions left
50 contexts left
50 responses left
50 questions left
49 contexts left
49 responses left
49 questions left
48 contexts left
48 responses left
48 questions left
47 contexts left
47 responses left
47 questions left
46 contexts left
46 responses left
46 questions left
45 contexts left
45 responses left
45 questions left
44 contexts left
44 responses left
44 questions left
43 contexts left
43 responses left
43 questions left
42 contexts left
42 responses left
42 questions left
41 contexts left
41 responses left
41 questions left
40 contexts left
40 responses left
40 questions left
39 contexts left
39 responses left
39 question

# MCQ generator

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["***KEY***"] = os.getenv("***KEY***")
os.environ["***KEY***_ENV"] = os.getenv("***KEY***_ENV")
os.environ["***KEY***"] = os.getenv("***KEY***")


from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")



from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant, to provide understandable and accurate information about pharmacogenomics and drugs.
Respond with correct option for the question based on the context and information provided also provide explanation for choosing the correct option, along with citing the references. Don't make assumptions.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [68]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["***KEY***"] = os.getenv("***KEY***")
os.environ["***KEY***_ENV"] = os.getenv("***KEY***_ENV")
os.environ["***KEY***"] = os.getenv("***KEY***")


from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")  # 



from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant, to provide understandable and accurate information about pharmacogenomics and drugs. You have to respond to the multiple choice questions following the instruction as follows:

1. Read the Question and the 4 Options provided carefully.

Example for the Question:

{{
    "Question": "If a patient taking tacrolimus is found to be a CYP3A5 extensive metabolizer, what should be done to their starting dose?",
    "Options": {{
        "A": "Decrease the starting dose by half.",
        "B": "Increase the starting dose by 1.5 to 2 times.",
        "C": "Maintain the current starting dose.",
        "D": "Stop the medication immediately."
        }}
}}

2. Based on the context, give the probabilities of each option being correct,
3. Also give the Correct Option and Explanation for the correct option along with citing the references.
4. Structure the response in the JSON format as: 

Example format of response:

{{
    "Options with Probability percentage": {{
        "A": {{"Decrease the starting dose by half.", 78.34}},
        "B": {{"Increase the starting dose by 1.5 to 2 times.", 95.36}},
        "C": {{"Maintain the current starting dose.", 68.42}},
        "D": {{"Stop the medication immediately.", 54.13}},
        }}
    "Correct option": "B. Increase the starting dose by 1.5 to 2 times",
    "Explanation": "For a CYP3A5 extensive metabolizer, the starting dose of tacrolimus should be increased by 1.5 to 2 times the recommended starting dose because they have a lower probability of achieving target concentrations",
    "References": ["CPIC Guidelines for Tacrolimus and CYP3A5: CPIC Guideline (https://cpicpgx.org/guidelines/guideline-for-tacrolimus-and-cyp3a5/)", "DPWG Guidelines: DPWG May 2020 Update (https://www.knmp.nl/downloads/pharmacogenetic-recommendations-may-2020.pdf)"]
}}


5. Do not provide anything outside the JSON structure. 

Use the context and question provided below for the response (Do not make assumptions or hallucinate):

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Consider prompt w/0 probability


In [16]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

folder_path = "/home/akkey/Akshay/RAG/RAG_data/all_files"
jsondata = []
csvdata = []
pdfdocs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        doc = loader.load()
        pdfdocs.extend(doc)
    elif filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        loader = CSVLoader(file_path)
        data = loader.load()
        csvdata.extend(data)
    elif filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        loader = JSONLoader(file_path, jq_schema=".",json_lines=False,text_content=False)
        data = loader.load()
        jsondata.extend(data)

for doc in pdfdocs:
    doc.page_content = doc.page_content.replace('\t', ' ')


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
documents = text_splitter.split_documents(pdfdocs)
jsondocs = text_splitter.split_documents(jsondata)

documents += jsondocs + csvdata

In [69]:
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers import BM25Retriever, EnsembleRetriever

pinecone = PineconeVectorStore(embedding=embeddings, index_name="doc-intel") # Chunk sizes

retriever_vectordb = pinecone.as_retriever(search_kwargs={"k": 25}) # retrieve less no., documents
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k =  25
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["COHERE_API_KEY"] = "zme42oLVhWXAAlxYc2bygeRB8wXWlJQ008awB9XY"

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=compression_retriever, question=RunnablePassthrough())


chain = setup | prompt | model | parser

In [64]:
import time
import numpy as np
import pandas as pd
Questions = pd.read_csv("/home/akkey/Akshay/RAG/json_mcq_questions.csv")
Questions

,Unnamed: 0,Questions,Correct Option,Explanation
0,0,{'Question': 'If a patient taking tacrolimus i...,{'Correct Option': 'B. Increase the starting d...,{'Explanation': 'For a CYP3A5 extensive metabo...
1,1,{'Question': 'How does having the HLA-B*15:02 ...,{'Correct Option': 'B. It restricts the use of...,{'Explanation': 'HLA-B*15:02 is associated wit...
2,2,{'Question': 'Which of the following is a pote...,{'Correct Option': 'C. Increased risk of Steve...,{'Explanation': 'Both allopurinol and phenytoi...
3,3,{'Question': 'Why should you avoid grapefruit ...,"{'Correct Option': ""C. It interferes with the ...",{'Explanation': 'Compounds in grapefruit can i...
4,4,{'Question': 'What can a genetic test determin...,"{'Correct Option': ""C. If the adverse reaction...","{'Explanation': ""A genetic test, specifically ..."
...,...,...,...,...
128,128,{'Question': 'If a patient has CYP2D6 poor met...,{'Correct Option': 'A. Granisetron'},{'Explanation': 'Granisetron is an alternative...
129,129,{'Question': 'What is the most common adverse ...,{'Correct Option': 'D. Diarrhea'},{'Explanation': 'The most common adverse effec...
130,130,{'Question': 'Which pharmacogenomic panels are...,"{'Correct Option': 'B. G6PD, CYP2D6, UGT1A1'}","{'Explanation': 'For patients with T2DM, hyper..."
131,131,{'Question': 'What is a safer alternative to c...,{'Correct Option': 'B. Sodium Valproate'},{'Explanation': 'Sodium Valproate is a safer a...


In [70]:
questions = Questions["Questions"]
questions

0      {'Question': 'If a patient taking tacrolimus i...
1      {'Question': 'How does having the HLA-B*15:02 ...
2      {'Question': 'Which of the following is a pote...
3      {'Question': 'Why should you avoid grapefruit ...
4      {'Question': 'What can a genetic test determin...
                             ...                        
128    {'Question': 'If a patient has CYP2D6 poor met...
129    {'Question': 'What is the most common adverse ...
130    {'Question': 'Which pharmacogenomic panels are...
131    {'Question': 'What is a safer alternative to c...
132    {'Question': 'What is the adverse outcome asso...
Name: Questions, Length: 133, dtype: object

In [60]:
n = len(questions)
i = 1
resp = []
cont = []

for question in questions:
    cont.append([docs.page_content for docs in compression_retriever.invoke(question)])
    Questions["Context"] = cont + [np.nan] * (n - i)
    time.sleep(60)
    resp.append(chain.invoke(question))
    Questions["Response"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(30)
    Questions.to_csv("/home/akkey/Akshay/RAG/mcq_gen/gpt-4o_json_responses.csv")
    i += 1

132 responses left
131 responses left
130 responses left
129 responses left
128 responses left
127 responses left
126 responses left
125 responses left
124 responses left
123 responses left
122 responses left
121 responses left
120 responses left
119 responses left
118 responses left
117 responses left
116 responses left
115 responses left
114 responses left
113 responses left
112 responses left
111 responses left
110 responses left
109 responses left
108 responses left
107 responses left
106 responses left
105 responses left
104 responses left
103 responses left
102 responses left
101 responses left
100 responses left
99 responses left
98 responses left
97 responses left
96 responses left


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [72]:
len(cont[:37])

37

In [73]:
n = len(questions)
i = 38
n-i

95

In [74]:
n = len(questions)
i = 38
resp = resp[:37]
cont = cont[:37]

for question in questions[37:]:
    cont.append([docs.page_content for docs in compression_retriever.invoke(question)])
    Questions["Context"] = cont + [np.nan] * (n - i)
    time.sleep(60)
    resp.append(chain.invoke(question))
    Questions["Response"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(30)
    Questions.to_csv("/home/akkey/Akshay/RAG/mcq_gen/gpt-4o_json_responses_2.csv")
    i += 1

95 responses left
94 responses left
93 responses left
92 responses left
91 responses left
90 responses left
89 responses left
88 responses left
87 responses left
86 responses left
85 responses left
84 responses left
83 responses left
82 responses left
81 responses left
80 responses left
79 responses left
78 responses left
77 responses left
76 responses left
75 responses left
74 responses left
73 responses left
72 responses left
71 responses left
70 responses left
69 responses left
68 responses left
67 responses left
66 responses left
65 responses left
64 responses left
63 responses left
62 responses left
61 responses left
60 responses left
59 responses left
58 responses left
57 responses left
56 responses left
55 responses left
54 responses left
53 responses left
52 responses left
51 responses left
50 responses left
49 responses left
48 responses left
47 responses left
46 responses left
45 responses left
44 responses left
43 responses left
42 responses left
41 responses left
40 respons

In [61]:
Questions_op = pd.read_csv("/home/akkey/Akshay/RAG/mcq_gen/gpt-4o_json_responses.csv")
Questions_op

,Unnamed: 0.1,Unnamed: 0,Questions,Correct Option,Explanation,Context,Response
0,0,0,{'Question': 'If a patient taking tacrolimus i...,{'Correct Option': 'B. Increase the starting d...,{'Explanation': 'For a CYP3A5 extensive metabo...,"['{""citations"": [], ""guideline"": {""objCls"": ""G...","{\n ""Options with Probability percentage"": ..."
1,1,1,{'Question': 'How does having the HLA-B*15:02 ...,{'Correct Option': 'B. It restricts the use of...,{'Explanation': 'HLA-B*15:02 is associated wit...,['describing the relationship between <em>HLA-...,"{\n ""Options with Probability percentage"": ..."
2,2,2,{'Question': 'Which of the following is a pote...,{'Correct Option': 'C. Increased risk of Steve...,{'Explanation': 'Both allopurinol and phenytoi...,['Drugs: phenytoin\nGenes: HLA-B*15:02\nPhenot...,"{\n ""Options with Probability percentage"": ..."
3,3,3,{'Question': 'Why should you avoid grapefruit ...,"{'Correct Option': ""C. It interferes with the ...",{'Explanation': 'Compounds in grapefruit can i...,"['{""guideline"": {""id"": ""PA166104948"", ""name"": ...","{\n ""Options with Probability percentage"": ..."
4,4,4,{'Question': 'What can a genetic test determin...,"{'Correct Option': ""C. If the adverse reaction...","{'Explanation': ""A genetic test, specifically ...",['NO action is needed for this gene-drug inter...,"```json\n{\n ""Options with Probability perc..."
...,...,...,...,...,...,...,...
128,128,128,{'Question': 'If a patient has CYP2D6 poor met...,{'Correct Option': 'A. Granisetron'},{'Explanation': 'Granisetron is an alternative...,NaN,NaN
129,129,129,{'Question': 'What is the most common adverse ...,{'Correct Option': 'D. Diarrhea'},{'Explanation': 'The most common adverse effec...,NaN,NaN
130,130,130,{'Question': 'Which pharmacogenomic panels are...,"{'Correct Option': 'B. G6PD, CYP2D6, UGT1A1'}","{'Explanation': 'For patients with T2DM, hyper...",NaN,NaN
131,131,131,{'Question': 'What is a safer alternative to c...,{'Correct Option': 'B. Sodium Valproate'},{'Explanation': 'Sodium Valproate is a safer a...,NaN,NaN


In [16]:
n = len(questions)
i = 105
resp = resp[:104].to_list()

for question in questions[104:]:
    response = chain.invoke(question)
    resp.append(response)
    Questions["responses"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(60)
    Questions.to_csv("/home/akkey/Akshay/RAG/mcq_gen/gpt-4o_3.csv")
    i += 1

29 responses left
28 responses left
27 responses left
26 responses left
25 responses left
24 responses left
23 responses left
22 responses left
21 responses left
20 responses left
19 responses left
18 responses left
17 responses left
16 responses left
15 responses left
14 responses left
13 responses left
12 responses left
11 responses left
10 responses left
9 responses left
8 responses left
7 responses left
6 responses left
5 responses left
4 responses left
3 responses left
2 responses left
1 responses left
0 responses left


In [18]:
resp = []

n = len(questions)
i = 1

for question in questions:
    response = chain.invoke(question)
    resp.append(response)
    Questions["responses"] = resp + [np.nan] * (n - i)
    print(f"{(n-i)} responses left")
    time.sleep(60)
    Questions.to_csv("/home/akkey/Akshay/RAG/mcq_gen/gpt-4o.csv")
    i += 1

104